<a href="https://colab.research.google.com/github/deekshakoul/LDA-Topic-modelling/blob/master/LDA_model_under_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import string
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

import spacy
nlp = spacy.load('en')


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
dataset_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
df = pd.DataFrame()
df["text"] = dataset_train.data
df["label"] = dataset_train.target
df["lname"] = df["label"].apply(lambda r : dataset_train["target_names"][r] )

Cleaning text

In [ ]:
def cleaning(text):
  text = re.sub('\S*@\S*\s?', '',text) #remove emails
  #text = re.sub('\s+', ' ', text) #new line #not required, handled during tokenisation
  #text = re.sub("\'", " ", text) #single quotes
  text= re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text) #remove links
  #translator = str.maketrans('','',string.punctuation) #!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
  #text = text.translate(translator) 
  return text;

from gensim.parsing.preprocessing import STOPWORDS

#stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're','do', 'nt', 'edu', 'use','could','would','should','say','one','know','get'])

In [ ]:
def lemmatization(text):
  word = gensim.utils.simple_preprocess(text, min_len=3,deacc=True) #list of tokens
  sent=[]
  text_p = ' '.join(word)
  doc = nlp(text_p)
  sent.extend([token.lemma_ for token in doc if token.pos_ not in ['PRON']and token.lemma_.islower()])
  sent = [token.strip() for token in sent] #remove extraspace around tokens
  sent = [token for token in sent  if token not in STOPWORDS] #remove stopwords
  return sent

In [ ]:
df['processed_text'] =df['text'] 
df['processed_text'] = df['processed_text'].apply(cleaning)
df['processed_text'] = df['processed_text'].str.lower()
#df['processed_text']
df['processed_text'] = df['processed_text'].apply(lemmatization)
df['processed_text']

0       [prakash, subject, pointer, cursor, article, a...
1       [rusty, sander, subject, server, multi, screen...
2       [gerard, driscoll, subject, xwindow, opaque, o...
3       [ralph, seguin, subject, numlock, mask, interf...
4       [dave, wilson, subject, xlib, timeout, summary...
                              ...                        
1769    [hudepohl, pmj, subject, search, phonetic, fon...
1770    [thomas, schlatter, subject, cool, bmp, file, ...
1771    [mike, tipton, subject, character, code, probl...
1772    [mark, whalley, subject, windows, background, ...
1773    [subject, novell, nick, fitzgerald, software, ...
Name: processed_text, Length: 1774, dtype: object

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary
'''
dictionary = gensim.corpora.Dictionary(df['processed_text'])
dictionary.filter_extremes(no_below=5, no_above=0.1, keep_n=13000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

In [ ]:
'''
Remove very rare and very common words:
- words appearing less than 15 times
- words appearing in more than 10% of all documents'''
#dictionary.filter_extremes(no_below=3, no_above=0.2, keep_n= 20000)

In [ ]:
len(dictionary.keys())

11807

In [ ]:
dictionary #: key: id, value: word
bow_corpus

In [ ]:
#LDA multicore 
'''
Training lda model using gensim.models.LdaMulticore 
'''
#TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 20, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 3)

In [ ]:
 lda_model =  gensim.models.ldamodel.LdaModel(passes=10,corpus=bow_corpus,id2word=dictionary,num_topics = 8)

In [ ]:
#finding best model by chnaging alpha,beta  and k
search_params = {
'decay' : [0.3,0.5,0.7],
'iterations' : [30, 50, 70],
'chunksize' : [1000, 2000, 3000]
}
#Most probable topics per word, per_word_topics: True

In [ ]:
import itertools
train = []
best_coherence_score = 0.0
best_log_perplexity = 0.0
c=0
for params in itertools.product(*search_params.values()):
  check = []
  check.append(params[0])
  check.append(params[1])
  check.append(params[2])
  lda_model =  gensim.models.ldamodel.LdaModel(corpus=bow_corpus,id2word=dictionary, update_every=1, chunksize=params[2],random_state=100,
                                             alpha='auto', eta='auto', decay= params[0], offset=1.0, eval_every=10, iterations= params[1], gamma_threshold=0.001, minimum_probability=0.01, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None)
  coherencemodel = CoherenceModel(model=lda_model,texts=df['processed_text'],corpus=bow_corpus,dictionary=dictionary, coherence='c_v')
  coherence_score = coherencemodel.get_coherence()
  check.append(coherence_score)   
  log_perplexity = lda_model.log_perplexity(bow_corpus)
  check.append(log_perplexity)

  if  coherence_score > best_coherence_score:
    best_coherence_score = coherence_score   
    best_lda_model_c = lda_model
    best_params = params   
  if  log_perplexity < best_log_perplexity:
    best_log_perplexity = log_perplexity   
    best_lda_model_p = lda_model
    best_params_preplexity = params   
  train.append(check)

  print(params, coherence_score)
         
param = ['decay','iterations','chunksize', 'coherence score', 'log perplexity']
df_model = pd.DataFrame(train, columns = param)
df_model


In [ ]:
coherencemodel = CoherenceModel(model=lda_model,texts=df['processed_text'],corpus=bow_corpus,dictionary=dictionary, coherence='c_v',topn=4)
coherence_score = coherencemodel.get_coherence()
coherence_score 

0.7267553358691677

In [ ]:
'''
Words and relative weight in each topic
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

In [ ]:
def per_topic_words(topw):
  topics_keywords = []
  for i in range(lda_model.num_topics):
    word_perlist=[]
    for j in range(topw):
      word_perlist.append(lda_model.show_topic(i,topw)[j][0])
    topics_keywords.append(word_perlist)
  return topics_keywords  
topic_keywords = per_topic_words(10)
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['W '+str(i) for i in range(df_topic_keywords.shape[1])]
#df_topic_keywords.index = ['T '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords  

,W 0,W 1,W 2,W 3,W 4,W 5,W 6,W 7,W 8,W 9
0,sound,long,patrick,mahan,lazarus,blaster,ghostscript,tony,boot,speaker
1,entry,output,section,printf,rule,build,char,info,stream,int
2,controller,xterm,andrew,ide,richard,newsreader,tin,institute,type,format
3,color,chz,rlk,monitor,air,yes,comp,sun,screen,server
4,server,client,comp,mit,start,paradox,course,manager,group,borland
5,max,bhj,giz,bxn,qax,nrhj,biz,tct,qtm,fpl
6,hard,controller,jumper,floppy,bio,pin,master,board,rom,slave
7,bmp,mydisplay,video,ftp,refresh,robert,ram,david,unix,diamond
8,key,xterm,ini,xfree,keyboard,sys,config,start,date,unix
9,swap,monitor,motherboard,buy,video,ram,upgrade,vga,diamond,price


In [ ]:
#Get the topic distribution for the given document, #doc in bow format
df_doc_topic = pd.DataFrame(columns = ['Doc_ID', 'Dominant_Topic', 'Prob_Per_Topic', 'Keywords', 'Doc_Text'])
for i in range(len(bow_corpus)):
  a = lda_model.get_document_topics(bow_corpus[i])
  a.sort(key = lambda x: x[1], reverse=True) #highest prob topic first
  topic_wp = lda_model.show_topic(a[0][0]) #show top 10 words byb default
  keywords = [w for (w,p) in topic_wp]
  row = [i, a[0][0], a[0][1], keywords,  df['processed_text'][i]]
  df_doc_topic.loc[i] = row

In [ ]:
df_doc_topic.head(10)

,Doc_ID,Dominant_Topic,Prob_Per_Topic,Keywords,Doc_Text
0,0,17,0.964815,"[motif, list, manager, sun, object, key, accel...","[prakash, subject, pointer, cursor, article, a..."
1,1,11,0.963255,"[screen, display, server, multi, mode, video, ...","[rusty, sander, subject, server, multi, screen..."
2,2,17,0.297186,"[motif, list, manager, sun, object, key, accel...","[gerard, driscoll, subject, xwindow, opaque, o..."
3,3,19,0.983621,"[unit, link, cross, gateway, error, gfx, day, ...","[ralph, seguin, subject, numlock, mask, interf..."
4,4,18,0.597262,"[font, printer, print, type, character, box, s...","[dave, wilson, subject, xlib, timeout, summary..."
5,5,0,0.449688,"[sound, long, patrick, mahan, lazarus, blaster...","[andre, beck, subject, need, shade, program, e..."
6,6,18,0.473078,"[font, printer, print, type, character, box, s...","[christian, rank, subject, xsun, run, sparccla..."
7,7,17,0.564787,"[motif, list, manager, sun, object, key, accel...","[marc, kraker, newman, subject, unsubscribe, o..."
8,8,2,0.655999,"[controller, xterm, andrew, ide, richard, news...","[chun, hang, lin, subject, xman, source, nntp,..."
9,9,3,0.787368,"[color, chz, rlk, monitor, air, yes, comp, sun...","[christian, mollard, subject, look, david, smy..."


In [ ]:
#lda_model.show_topic(5, topn=10)
df_doc_topic.iloc[df_doc_topic[df_doc_topic["Dominant_Topic"]==5].index]

,Doc_ID,Dominant_Topic,Prob_Per_Topic,Keywords,Doc_Text
1193,1193,5,0.998480,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1208,1208,5,0.998713,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1209,1209,5,0.998475,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1222,1222,5,0.999028,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1294,1294,5,0.998506,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1297,1297,5,0.998126,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1335,1335,5,0.999092,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1406,1406,5,0.998547,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1620,1620,5,0.998964,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."
1759,1759,5,0.997532,"[max, bhj, giz, bxn, qax, nrhj, biz, tct, qtm,...","[subject, roman, bmp, cliff, reply, cliff, dis..."


In [ ]:
lda_model.show_topic(11, topn=20)

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_doc_topic.groupby('Dominant_Topic').agg('count')
topic_counts.reset_index(level=0,inplace=True)
topic_counts = topic_counts[['Dominant_Topic','Keywords']].rename(columns= {'Dominant_Topic':'Topic','Keywords':'Total Docs'})
topic_counts['Keywords']=topic_counts['Topic'].apply(get_keyword)
topic_counts.sort_values('Total Docs',ascending=False)

,Topic,Total Docs,Keywords
15,15,51,"long, xdm, patrick, mahan, lazarus, dortmund, ..."
14,14,47,"xfree, faq, multi, list, svr, resource, mouse,..."
16,16,44,"compile, gcc, libx, library, build, link, icon..."
13,13,37,"tar, export, contrib, pub, base, ftp, offer, i..."
11,11,37,"visual, key, apr, keycode, date, default, rece..."
10,10,34,"slow, xputimage, memory, box, animation, netwo..."
6,6,34,"course, member, year, mix, systems, disk, prog..."
7,7,34,"list, linux, athena, people, mailing, bsd, gad..."
3,3,32,"pixmap, depth, fail, request, number, colormap..."
5,5,31,"colormap, key, accelerator, memory, resource, ..."


In [ ]:
def get_keyword(topic):
  topic = int(topic)
  wp = lda_model.show_topic(topic)
  topic_keywords = ", ".join([word for word, prop in wp])
  return topic_keywords

In [ ]:
#print(df['processed_text'][144])
print(df['text'][1759])
#print(df['lname'][1620])
#Example of document which contains weird data

Subject: roman.bmp 05/14
From: pwiseman@salmon.usd.edu (Cliff)
Reply-To: pwiseman@salmon.usd.edu (Cliff)
Distribution: usa
Organization: University of South Dakota
Lines: 957


------------ Part 5 of 14 ------------
M)B9R&KJZ)EVZ=+ITNKH>?7U=PGU]+"Q=?72_="PU'GT>+,<LORPU86$1EX,8
M,P$HWA=@8!SKWI_,S"B<)S<W-R>!5C<-S`HFNAIR<K)R<B9RMW)+D)#D"C!V
M=KPYWA>\'.L74?4<JPW,5EQEFYJLKHBQ?K5^(7*ZLK*Z)KH:NKJZ)G0FNKI=
M)K)=)KHFNGU=NK*RLB9TLKIT)L)=PK)=71JRNKH:<AIR&KH:<B9R&G*W<G*W
M&K>W&JVM<G)RK:UR<K>W<JVM<G)R<A^M<G*WMW(5<K=RK:URMW)RM[>WM[=R
M<JUR'Q^W<A45<JUR%:VMMQ\5K7)RK16WMQ6MK:T5K:UO%6\5'Q6MK6\5K6]O
M%6^M%:T5;Q6W%16M%:VW%:T?M[>M%:VWK:VM<K<5K4]RK:UR<G)RMW*W3W*M
MMW(:<B9R<B8:MW)R<G*W&AH:&K<:<K=R<JURK7*MMW*M<G*W%;>MK16MMZT(
M"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("*T5K145MZVM<A45K;<(
M"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("!H:<AIR<G)R
M<B8F<G(:<G)R<G(:K7)R<G)/3[=RK7*W'ZUR%;>MK:VW;[<5%6\5%15O;Q45
M;V]O%15OJ*AO%6]O;ZBHJ&\5J`@("`@("`@("`@("`@("`@("`@("`@("`@(
M"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@